In [ ]:
"""
tournaments.ipynb

Jupyter Notebook to perform analysis on card market data.

Author: Casey Forey
Date Created: 4/7/24
"""

In [ ]:
%matplotlib inline

# Imports
from importlib import reload
import json 
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import src.constants as c
import src.load.load_set_data as lsd
import src.load.load_tournament_data as ltd
import src.plot.plot_set_data as psd
import src.plot.plot_tournament_data as ptd

In [ ]:
path = 'data/AtomicCards.json' 
f = open(path, encoding= "utf8") 
atm_cards = json.load(f)

path = 'data/AllPrintings.json' 
f = open(path, encoding= "utf8") 
all_cards = json.load(f)

path = 'data/AllPricesToday.json' 
f = open(path) 
prices = json.load(f)

path = 'data/SetList.json' 
f = open(path, encoding= "utf8") 
set_list = json.load(f)

## Modern Data

In [ ]:
base_path: str = os.path.join(c.DATA_DIRECTORY, '2023')
modern_card_counts_df, player_counts_df = ltd.load_format_card_counts(base_path, 'modern')
modern_card_counts_df

In [ ]:
data = atm_cards['data']
cards_set = {}
i = 0
for card in data:
    card_info = data[card][0]
    if 'firstPrinting' in card_info:
        cards_set[card] = card_info['firstPrinting']
    else:
        i +=1
print(i)

In [ ]:
cards_set_df = pd.DataFrame.from_dict(cards_set, orient='index')
cards_set_df.reset_index(inplace= True) 
cards_set_df.rename(columns={'index': 'card_name', 0: 'set_code'},inplace= True) 
cards_set_df

In [ ]:
def add_back_face(df):
    for card in df['card_name']:
        for name in cards_set_df['card_name']:
            front_name = name.split('//')[0].strip()
            if card == front_name and '//' in name:
                df['card_name'].replace(card, name,inplace = True)
    return df

In [ ]:
modern_card_counts_df = add_back_face(modern_card_counts_df)

In [ ]:
# Checking to see if double face cards look correct
for name in modern_card_counts_df['card_name']:
    if "Smelt" in name:
        print(name)

In [ ]:
modern_card_counts_df[modern_card_counts_df['card_name'] == 'Smelt // Herd // Saw']

In [ ]:
modern_card_counts_df = modern_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
modern_card_counts_df

In [ ]:
reload(lsd)

set_release_df: pd.DataFrame = lsd.get_set_and_release_year()
set_release_df

In [ ]:
modern_card_counts_df = modern_card_counts_df.merge(set_release_df, on='set_code', how='left', copy=True)
modern_card_counts_df

In [ ]:
reload(psd)
%matplotlib inline

legend_params: list[tuple] = [
    (1993, 'green', '--', 'Magic First Comes Out (1993)'),
    (2011, 'orange', '--', 'Start of Modern Format (2011)'),
    (2019, 'red', '--', 'Fire Design Principle Implemented (2019)'),
    (2021, 'blue', '--', 'Modern Horizons 2 Released (2021)'),
]
psd.plot_stacked_set_counts(modern_card_counts_df, set_release_df, 'modern', True, legend_params)

In [ ]:
# Very limited Graph as a proof of concept
modern_card_counts_df.groupby('release_year')['total_count'].sum().plot(kind='bar')

In [ ]:
# reload(ptd)

# ptd.plot_card_counts(modern_card_counts_df,(255/255,122/255,14/255))

### Pioneer Data

In [ ]:
base_path: str = os.path.join(c.DATA_DIRECTORY, '2023')
pioneer_card_counts_df, pioneer_player_counts_df = ltd.load_format_card_counts(base_path, 'pioneer')
pioneer_card_counts_df

In [ ]:
pioneer_card_counts_df = add_back_face(pioneer_card_counts_df)
pioneer_card_counts_df 

In [ ]:
for name in pioneer_card_counts_df['card_name']:
    if "Fable" in name:
        print(name)

In [ ]:
pioneer_card_counts_df = pioneer_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
pioneer_card_counts_df

In [ ]:
pioneer_card_counts_df = pioneer_card_counts_df.merge(set_release_df, on = 'set_code', how = 'left', copy = True)
pioneer_card_counts_df

In [ ]:
reload(psd)
%matplotlib inline

legend_params: list[tuple] = [
    (1993, 'green', '--', 'Magic First Comes Out (1993)'),
    (2019, 'red', '--', 'Fire Design Principle Implemented (2019)'),
    (2019, 'blue', ':', 'Pioneer Format Starts (2019)'),
]
psd.plot_stacked_set_counts(pioneer_card_counts_df, set_release_df, 'pioneer', True, legend_params)

In [ ]:
# reload(ptd)

# ptd.plot_card_counts(pioneer_card_counts_df, (9/255, 121/255, 105/255))

## Legacy

In [ ]:
base_path: str = os.path.join(c.DATA_DIRECTORY, '2023')
legacy_card_counts_df, legacy_player_counts_df = ltd.load_format_card_counts(base_path, 'legacy')
legacy_card_counts_df

In [ ]:
legacy_card_counts_df = add_back_face(legacy_card_counts_df)
legacy_card_counts_df

In [ ]:
for name in legacy_card_counts_df['card_name']:
    if "Fable" in name:
        print(name)

In [ ]:
legacy_card_counts_df = legacy_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
legacy_card_counts_df

In [ ]:
legacy_card_counts_df = legacy_card_counts_df.merge(set_release_df, on = 'set_code', how = 'left', copy = True)
legacy_card_counts_df

In [ ]:
reload(psd)
%matplotlib inline

legend_params: list[tuple] = [
    (1993, 'green', '--', 'Magic First Comes Out (1993)'),
    (1997, 'blue', ':', 'Legacy Format Starts (1997)'),
    (2019, 'red', '--', 'Fire Design Principle Implemented (2019)'),
]
psd.plot_stacked_set_counts(legacy_card_counts_df, set_release_df, 'legacy', True, legend_params)

In [ ]:
# reload(ptd)

# ptd.plot_card_counts(legacy_card_counts_df, (255/255, 215/255, 0/255))